# Snake-RL (Pytorch)
Gait Decomposition을 활용하여 Gait 자체를 학습하는 연구를 이 Jupyter 노트북에 작성을 하려고함. Gait Decomposition은 뱀 로봇의 움직임을 P함수와 M행렬로 분해하여 뱀 로봇의 Gait를 표현하는 방법이며, 이 방법을 통해서 직관적으로 뱀 로봇의 움직임을 표현할 수 있고, 파라미터 튜닝을 진행할 수 있다. 관련 내용은 [Arxiv 논문](https://arxiv.org/abs/2112.02057)을 참조할 것.

## Gait Decomposition에 대한 이해
Gait Decomposition에 대한 코드는 Gait 클래스에 구현되어 있다. 따라서 Gait 생성 코드를 코드에 import하자.

In [1]:
import numpy as np
import math

from gait import gait

g = gait(1)

위 코드에서 삽입된 gait.py코드는 위 논문의 P함수와 M행렬을 객체로 구현한 코드이다.

Gait가 잘 생성되는지 확인해보자.

In [2]:
for i in range(0,10):
    print(g.generate(i).T)

[[15.  0. -0. -0.  0.  0. -0. -0.  0.  0.  0.  0. -0. -0.]]
[[ 0.   13.97 -0.   -0.    0.    0.   -0.   -0.    0.    0.   -0.   -0.
  -0.   -0.  ]]
[[  0.     0.   -23.31  -0.     0.     0.    -0.    -0.     0.     0.
   -0.    -0.    -0.    -0.  ]]
[[  0.     0.    -0.   -24.04   0.     0.    -0.    -0.     0.     0.
   -0.    -0.    -0.    -0.  ]]
[[ 0.    0.   -0.   -0.   28.53  0.   -0.   -0.    0.    0.   -0.   -0.
  -0.   -0.  ]]
[[ 0.    0.   -0.   -0.    0.   29.42 -0.   -0.    0.    0.   -0.   -0.
  -0.   -0.  ]]
[[  0.     0.    -0.    -0.     0.     0.   -29.96  -0.     0.     0.
   -0.    -0.    -0.    -0.  ]]
[[ -0.     0.    -0.    -0.     0.     0.    -0.   -29.08   0.     0.
   -0.    -0.     0.    -0.  ]]
[[-0.    0.   -0.   -0.    0.    0.   -0.   -0.   27.41  0.   -0.   -0.
   0.   -0.  ]]
[[-0.    0.   -0.   -0.    0.    0.   -0.   -0.    0.   23.07 -0.   -0.
   0.   -0.  ]]


위와 같이 생성된 Gait 데이터를 통해서 모터를 동작시키면 뱀 로봇이 움직이게 된다. Mujoco 시뮬레이터에서 움직이는 것을 그림으로 나타내면 아래 그림과 같다.

![fig1](./img/fig1.gif)

위 출력 결과와 같이 $i$가 증가하면서 각 모터에 입력되는 입력 신호가 바뀌는 것을 확인할 수 있다. 
위 결과는 P함수와 M행렬의 행렬 곱에서 대각 원소의 값을 출력한 결과이다. 위와 같은 결과를 모터에 전달하여 뱀 로봇이 움직이도록 만들 수 있다.

P함수는 모터의 목표 위치를 계산하는 함수로 주기 함수의 꼴을 갖는다. 주변 환경이 변하지 않는 이상적인 환경에서는 미리 학습된 P함수를 통해서 뱀 로봇을 움직이게 만들 수 있다.
하지만, 뱀 로봇이 항상 이상적인 환경에서 동작하지 않기 때문에 학습된 P함수가 최적 값을 내지못할 수도 있다. 우리는 이런 점에서 P함수를 센싱되는 지형의 데이터나 로봇의 상태에 따라서 변경할 수 있는 알고리즘을 구현하고자 한다.

이를 위해 P함수의 정의를 명확하게 하고자한다.

$$
P(t,A,\phi) \to \bar{\theta}
$$

위 식을 확인하면 P함수는 시간과 진폭, 위상차를 독립 변수로 갖는 함수이다. 더 상세하게 P함수를 나타내면 다음과 같다.

$$
\begin{equation*}
    \bar{\theta} = \left( 
    \begin{matrix}
    i = \text{odd}\;, \theta_i =  A_{\text{dor.}}\cdot \sin(\frac{2 \pi}{m}t + \frac{i}{2} \cdot \phi_{\text{dor.}})\;\;\; \\ \\
    i = \text{even}\;, \theta_i =   A_{\text{lat.}}\cdot \sin(\frac{2 \pi}{m}t + \frac{i-1}{2}\cdot \phi_{\text{lat.}})\; \end{matrix} 
    \right)\quad\quad
\end{equation*}
$$ 
여기에서, $m$은 P함수의 주파수 성분을 나타내기 위한 상수이며, $i$는 모터의 순서를 나타낸다.  우리 시스템에서는 Head 모터가 0번이다. $(i=0, \text{Head})$

이 때 다시 Gait 생성 코드를 보면 순서대로 머리모터부터 홀수 번째 모터들이 순서대로 움직이는 것으로 볼 수 있다. P함수는 t에 따라서 모든 모터에 대한 $\theta$값이 변경되는데 홀수번째 모터만 동작하는 이유는 우리가 뱀 로봇의 Gait를 정의하기 위해서 모션 행렬 M을 정의했기 때문이다. 모션 행렬 M은 뱀 로봇의 Gait를 구분하기 위한 주요한 파라미터이다. 이 행렬을 통해서 시간 t에서 어떤 모터가 동작할지 정할 수 있다. 현재 우리는 뱀 로봇의 Gait로 3가지 Gait를 정의했다. Inchworm, Serpentine, Sidewind이고 이를 모션 행렬로 표현하면 아래와 같다.

### Inchworm gait의 모션 행렬
$$
\begin{equation*}
    \mathbb{M_\text{inch}} = \begin{bmatrix}
1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
0 & 0 & 0 & 0 & 0 & 0 & 1 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 \\
\end{bmatrix} \end{equation*}
$$
### Serpentine gait의 모션 행렬
$$
\begin{equation}
    \mathbb{M_\text{ser}} = \begin{bmatrix}
1 & 0 & 0 & \cdots & 0 & 0\\
0 & 1 & 0 & \cdots & 0 & 0\\
0 & 0 & 1 & \cdots & 0 & 0\\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
 0 & 0 & 0 & \cdots & 1 & 0\\
 0 & 0 & 0 & \cdots & 0 & 1 
\end{bmatrix} = \mathbb{I}
\end{equation} 
$$
### Sidewind gait의 모션 행렬
$$
\begin{equation}
    \mathbb{M_\text{side}} = \begin{bmatrix}
0 & 1 & 0 & \cdots & 0 & 0 \\
1 & 0 & 0 & \cdots & 0 & 0 \\
0 & 0 & 0 & \cdots & 0 & 0 \\
0 & 0 & 1 & \cdots & 0 & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
0 & 0 & 0 & \cdots & 0 & 1\\
0 & 0 & 0 & \cdots & 1 & 0\\
\end{bmatrix}
\end{equation}
$$

$\mathbb{m}$행렬은 k개의 열벡터로 이루어진다. $(\mathbb{M} = \{m_1,..m_k\})$ 여기에서 이 열벡터는 해당 시간에서 움직이는 모터의 인덱스를 표현한다. 따라서 P함수와 M행렬을 내적하면 어떤 모터가 어느 정도 움직이면 되는지를 표현할 수 있다. 여기에서 시스템 모터의 목표 각도 Array를 $\mathbb{G}$라고 가정하면, 수식으로 다음과 같이 표현할 수 있다.

$$
\begin{equation*}
\mathbb{M} = \left[
\begin{matrix}
    \vert & \vert &        & \vert \\
    m_1   &   m_2 & \cdots & m_k \\
    \vert & \vert &        & \vert \\
\end{matrix}
\right] \; , \, m_k \in \mathbb{R}^{14}\\
\end{equation*}
$$

$$
\begin{equation*}
    \mathbb{G}_k = \bar{\theta} \, \cdot \, {m_k}^{T}   \; , \; \mathbb{G}_{k} \in \mathbb{R}^{14 \times 14}
\end{equation*}
$$

이렇게 표현된 Gait는 P함수의 파라미터를 조절하여 튜닝(최적화)시킬 수 있다. 최적화를 진행하기 위해서는 Gait의 성능을 평가할 수 있는 범함수가 필요하다. 우리는 성능 평가를 위한 범함수를 아래와 같이 정의했다.

$$
\begin{equation}
    U(k,P(k,\,\bar{\theta}),\mathbb{M}) = {i} \cdot \Delta x - {j} \cdot |\Delta y| - {l} \cdot |\frac{\Delta y}{\Delta x}| - m \cdot {\int}_{t_{0}}^{t_f} {\| \mathbb{G}_{k} \|}_1 \,dk \\
    \\
\end{equation}
$$

위 범함수는 정의된 뱀 로봇이 10초 동안 움직이고 난 뒤의 결과를 통해 Gait 성능을 평가하는 함수이다. $i,j,l$은 성능 지표 이며 이 지표를 변경하면서 어떤 성능을 중점적으로 최적화할 것인지 표현할 수 있다. 위와 같은 범 함수를 활용하여 P함수의 파라미터를 최적화시킬 수 있었다. 하지만 이렇게 최적화된 Gait는 Open-loop로 제어되어, 뱀 로봇의 상태를 피드백받지 않고 모터의 목표값을 생성한다. 따라서 다양한 환경에서 적응하기 어렵다. 

우리는 이렇게 Decomposed된 Gait 구성 요소를 활용하여 Closed-loop제어를 하기위해서 DNN 네트워크를 활용하고자 한다. 

제안하는 네트워크 구조는 다음과 같다.





## State Listener 구현
위와 같은 구조로 뱀 로봇을 움직이기 위해서 시뮬레이션 결과를 받아올 수 있는 Listener를 구현해야 했다. 신경망 입력으로는 벡터 $\bar{x}$가 입력되며 이 벡터의 원소는 아래와 같다.

$$
\bar{x} = \left( \begin{align*} k \\ \theta \\ \dot{\theta} \\ \bar{q} \end{align*}\right)
$$

여기서 $k$는 time step을 나타내며, $\theta$는 관절의 각도, $\bar{q}$는 머리 링크의 오리엔테이션과 위치를 나타낸다.

이와 같은 데이터를 Mujoco 시뮬레이터에서 얻어야한다. Python에서 Mujoco를 import하고 관련된 설정을 진행한다.

In [3]:
import mujoco_py

snake = mujoco_py.load_model_from_path("../description/mujoco/snake_dgist.xml")
simulator = mujoco_py.MjSim(snake)

위와 같은 설정을 통해서 뱀 로봇 정의 파일을 Mujoco가 불러와 시뮬레이션 환경을 구축할 수 있게 되었다.

이후 각 Step마다 필요한 정보를 가져오는 함수를 구현하면 아래와 같다.

In [4]:
def stepListener(k) -> np.ndarray:
    joint_names = ['joint1','joint2','joint3','joint4','joint5','joint6','joint7','joint8','joint9','joint10','joint11','joint12','joint13','joint14']

    theta = np.array([simulator.data.get_joint_qpos(x) for x in joint_names])
    dtheta = np.array([simulator.data.get_joint_qvel(x) for x in joint_names])
    x = np.array(simulator.data.get_body_xpos('head'))
    q = np.array(simulator.data.get_body_xquat('head'))

    # print(theta)
    # print(dtheta)
    # print(q)
    state = np.concatenate((np.array([k]),theta,dtheta,x,q))

    # return np.array([k, theta, dtheta, q],dtype='object')
    return state

## Reward 계산 함수 구현
받아온 State를 통해서 Reward를 계산하는 함수를 구현해야한다. GD 최적화 논문에서는 0초에서 Terminal까지의 변위를 통해 Reward를 계산했지만 여기서는 State간의 Reward를 계산해야한다. 이를 나타낸 코드는 아래와 같다.

In [5]:
def getReward(l_s:list, c_s:list)-> float:
    l_s = np.array(l_s)
    c_s = np.array(c_s)

    s = c_s - l_s

    reward = 1500 * s[-7] - 800 * abs(s[-6]) - 900 * abs(s[-6] / s[-7])

    return reward

이후 뱀 로봇이 P함수와 M행렬에 따라서 움직일 수 있도록 $\mathbb{G}$벡터를 계산하는 코드를 구현해야 한다. Gait Decomposition은 Gait 파라미터가 바뀌지 않는 상태에서 연속적으로 뱀 로봇의 움직임을 생성하는 반면에, 신경망을 통해 Feed forward하는 이 시스템의 경우 새롭게 Gait 파라미터를 받아 P함수의 특성을 바꾸는 기능이 추가되어야한다. 이를 코드로 구현하면 아래와 같다.

In [6]:
def genGait(k:int, gait:int, d_amp:float, d_phase:float, d_lam:float,l_amp:float, l_phase:float, l_lam:float, tau:int) -> np.ndarray:
    if tau < 1:
        tau = 1
    g.setParams(gait, d_amp, d_phase, d_lam, l_amp, l_phase, l_lam, tau)
    goal_P = g.generate(k)

    return goal_P

위와 같이 구현한 코드가 시뮬레이터에서 잘 작동하는지 확인해보도록 하자. 아직 신경망이 구현되지 않았으므로 임의의 Gait 파라미터를 생성하기 위해서 Random 패키지를 사용하여 Gait를 생성하기로 한다. 이를 코드로 구현하면 아래와 같다.

In [7]:
import random

# viewer = mujoco_py.MjViewer(simulator) #For rendering sim results.

# for k in range(0,5000):
#     p1 = random.randint(0,900)/10
#     p2 = random.randint(0,3600)/10
#     p3 = random.randint(-10,10)
#     p4 = random.randint(0,900)/10
#     p5 = random.randint(0,3600)/10
#     p6 = random.randint(-10,10)
#     p7 = random.randint(1,4)

#     G_k = genGait(k,1,p1,p2,p3,p4,p5,p6,p7)
#     motor_idx = np.nonzero(G_k)

#     for idx in motor_idx:
#                 if not(len(idx) == 0):
#                     simulator.data.ctrl[idx] = g.degtorad(G_k[idx])

#     simulator.step()
#     viewer.render()


위 코드를 실행시키면 아래와 같은 모양으로 뱀 로봇이 움직이는 것을 확인할 수 있다.
![fig2](./img/randomized-movement.gif)

## Replay Memory 구현
Replay Memory는 학습 속도를 증가시키기 위해서 여러 케이스로 실험을 진행한 결과를 저장할 수 있도록 구현된 Memory이다. 일반적으로 학습에는 병렬 연산이 많이 이루어지므로 데이터를 모아 둔 뒤 한번에 학습하는 것이 더 빠르다고 한다. Replay Memory코드는 [Pytorch 튜토리얼](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html)을 참고했다.



In [8]:
from collections import namedtuple, deque

Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

## Q-Network 구현
뱀 로봇의 상태 변화를 통해서 적절한 제어 신호를 생성하는 Q Network는 그 신경망 구조에 따라서 학습 속도나 성능이 달라질 수 있다. 우선 간단한 MLP구조의 신경망을 통해서 뱀 로봇의 제어를 진행해보도록 코드를 구현했다.

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class DQN_MLP(nn.Module):
    def __init__(self):
        super(DQN_MLP, self).__init__()

        self.fc1 = nn.Linear(36,60, bias=True)
        self.fc2 = nn.Linear(60,30, bias=True)
        self.fc3 = nn.Linear(30,8,bias=True)
        self.drop = nn.Dropout(0.15)

    def forward(self, x):
        x = x.to(device)
        # x = torch.tensor(x,device=device)
        x = x.float()
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.dropout(x)
        x = F.relu(self.fc3(x))

        return x


MLP 기반으로 구성된 Q-Network는 3개의 Layer가 있으며 각 레이어가 선형 결합되도록 구성되었다. 또한 2번 층 이후에 Dropout를 진행한다. 

## Training 루프 구현 준비
뱀 로봇을 움직이게 만들고, 이를 학습시키기 위한 준비가 끝났다. 본격적으로 신경망을 학습시키는 코드를 구현해야한다. 먼저 학습에 필요한 Hyper 파라미터를 조절하는 코드와 액션 벡터를 추출하는 함수 등을 구현한다..

### Hyper 파라미터
Hyper 파라미터는 단순한 변수로 표현할 수 있다. 이에 해당되는 코드는 아래와 같다.

In [10]:
BATCH_SIZE = 4
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

# n_actions = 14 #number of joints

policy_net = DQN_MLP().to(device)
target_net = DQN_MLP().to(device)

target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.Adam(policy_net.parameters())

memory = ReplayMemory(10000)


### Action 추출기 구현

구현된 신경망을 통해서 다음 State로 전이되기 위한 Action을 추출하는 코드를 아래와 같이 구현했다.

In [11]:
steps_done = 0

def select_action(input):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)

    steps_done = steps_done + 1

    if sample > eps_threshold:
        with torch.no_grad():
            return policy_net(input)
    else:
        return torch.tensor([random.randrange(-80,80), 
        random.randrange(-360,360), 
        random.randrange(-10,10), 
        random.randrange(-80,80), 
        random.randrange(-360,360), 
        random.randrange(-10,10), 
        random.randrange(1,7), 
            ], device=device)

### 학습 결과 Plot 함수 구현
에피소드 마다 학습되는 결과를 Plot하기 위한 Plotting 함수를 구현하면 아래와 같다.

In [12]:
episode_durations = []

import matplotlib
import matplotlib.pyplot as plt

def plot_duration():
    plt.figure(2)
    plt.clf()
    duration_t = torch.tensor(episode_durations, dtype=torch.float)

    plt.title('Training process...')
    plt.xlabel('Episodes')
    plt.ylabel('Duration')
    plt.plot(duration_t.numpy())

    if len(duration_t) >= 100:
        means = duration_t.unfold(0,100,1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))

        plt.plot(means.numpy())

    plt.pause(0.01)
    
    if is_ipython:
        display.clear_output(wait=True)
        display.display(plt.gcf())

## Training Loop 구현
학습을 위해 필요한 대부분의 블록이 구현되었다. 이제 실제로 학습을 진행하는(Optimizing) 코드를 구현해야한다. 이는 아래와 같다.

In [13]:
def optmize_model():
    if len(memory) < BATCH_SIZE:
        print('Sim data is too small terminate optimizing...')

    transition = memory.sample(BATCH_SIZE)

    batch = Transition(*zip(*transition))

    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)), device=device, dtype=torch.bool)

    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
    non_final_next_states = torch.reshape(non_final_next_states,(BATCH_SIZE,36))

    state_batch = torch.cat(batch.state)
    state_batch = torch.reshape(state_batch,(BATCH_SIZE,36))
    action_batch = torch.cat(batch.action)
    action_batch = torch.reshape(action_batch,(BATCH_SIZE,8))
    reward_batch = torch.cat(batch.reward)
    reward_batch = torch.reshape(reward_batch,(BATCH_SIZE,1))

    gather_indices = [7 for i in range(BATCH_SIZE)]
    gather_indices = torch.tensor(gather_indices,device=device).unsqueeze(axis=-1)
    
    state_action_values = policy_net(state_batch)
    state_action_values = torch.gather(state_action_values,1,gather_indices)

    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).gather(1,gather_indices).reshape(BATCH_SIZE).detach()

    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values,expected_state_action_values.unsqueeze(1))

    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1,1)
    optimizer.step()

## Main Loop 구현
위 코드들을 이용해서 Sim Data를 획득하고 최적화를 진행하는 코드를 구현하면 아래와 같다.

In [15]:
num_episodes = 20

viewer = mujoco_py.MjViewer(simulator) #For rendering sim results.

for i_episode in range(num_episodes):
    simulator.reset()
    k = 0
    last_state = stepListener(k)

    simulator.step()

    for k in range(300):
        # action = select_action(torch.from_numpy(last_state))
        action = policy_net(torch.from_numpy(last_state))

        gait_param = action.tolist()

        G_k = genGait(k,1, gait_param[0], gait_param[1], gait_param[2], gait_param[3], gait_param[4], gait_param[5], gait_param[6])

        motor_idx = np.nonzero(G_k)

        for idx in motor_idx:
                    if not(len(idx) == 0):
                        simulator.data.ctrl[idx] = g.degtorad(G_k[idx])

        simulator.step()
        viewer.render()

        current_state = stepListener(k)
        reward = getReward(last_state, current_state)
        reward = torch.tensor([reward], device=device)
        
        last_state = current_state

        memory.push(torch.from_numpy(last_state), action, torch.from_numpy(current_state), reward)

        if k == 299:
            memory.push(_,_,None,_)

    optmize_model()

    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

Creating window glfw


/home/bong/anaconda3/envs/rl/lib/python3.8/site-packages/torch/nn/modules/loss.py:912: UserWarning: Using a target size (torch.Size([4, 1, 4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)


In [ ]:
transition = memory.sample(BATCH_SIZE)

batch = Transition(*zip(*transition))

non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)), device=device, dtype=torch.bool)

non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
non_final_next_states = torch.reshape(non_final_next_states,(BATCH_SIZE,36))

state_batch = torch.cat(batch.state)
state_batch = torch.reshape(state_batch,(BATCH_SIZE,36))
action_batch = torch.cat(batch.action)
action_batch = torch.reshape(action_batch,(BATCH_SIZE,8))
reward_batch = torch.cat(batch.reward)
reward_batch = torch.reshape(reward_batch,(BATCH_SIZE,1))

gather_indices = [7 for i in range(BATCH_SIZE)]
gather_indices = torch.tensor(gather_indices,device=device).unsqueeze(axis=-1)
state_action_values = policy_net(state_batch)
state_action_values = torch.gather(state_action_values,1,gather_indices)

next_state_values = torch.zeros(BATCH_SIZE, device=device)
next_state_values[non_final_mask] = target_net(non_final_next_states).gather(1,gather_indices).reshape(BATCH_SIZE).detach()


In [ ]:
target_net(non_final_next_states).gather(1,gather_indices).reshape(BATCH_SIZE).detach()